In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("seaborn-v0_8-darkgrid")
import seaborn as sns
import gudhi as gd

import gudhi.hera as hera
from gudhi.wasserstein import wasserstein_distance
import gudhi.representations as gr

import plotly.graph_objs as go
import plotly.io as pio
import plotly
import ipywidgets as widgets

from plotly.offline import iplot
import itertools

 INSPECT TOPOLOGICAL PROPERTIES OF TORUS:
 - CREATE THE PROPER POINT CLOUD
 - INIT AN ALPHA-COMPLEX or VIETORIS-RIPS COMPLEX
 - COMPUTE THE PERSISTENCE DIAGRAM
 - INSPECT THE TRIANGULARIZATION


In [ ]:
def generate_random_points_on_torus(num_points, major_radius, minor_radius):
    # Generate random angles for theta and phi
    theta = np.random.uniform(0, 2*np.pi, num_points)
    phi = np.random.uniform(0, 2*np.pi, num_points)
    
    # Convert spherical coordinates to Cartesian coordinates
    x = (major_radius + minor_radius * np.cos(phi)) * np.cos(theta)
    y = (major_radius + minor_radius * np.cos(phi)) * np.sin(theta)
    z = minor_radius * np.sin(phi)
    
    return np.stack((x, y, z), axis=-1)

# Define parameters of the torus
major_radius = 2.0
minor_radius = 1.0
num_points = 2000

# Generate random points on the torus
points_on_torus = generate_random_points_on_torus(num_points, major_radius, minor_radius)

print("Generated random points on the torus shape:", points_on_torus.shape)

In [ ]:
def scatter_torus(points, fig_title):
    # Create trace for the point cloud
    trace = go.Scatter3d(
        x=points[:, 0],
        y=points[:, 1],
        z=points[:, 2],
        mode='markers',
        marker=dict(
            size=1.5,
            color='red',
            opacity=0.7
        )
    )

    # Create layout
    layout = go.Layout(
        scene=dict(
            xaxis=dict(title='X'),
            yaxis=dict(title='Y'),
            zaxis=dict(title='Z')
        ),
        margin=dict(l=0, r=0, b=0, t=0)
    )

    # Create figure
    fig = go.Figure(data=[trace], layout=layout)

    # Show interactive plot
    fig.update_layout(
        # template="seaborn",  # Alternative to "plotly_white"
        scene=dict(
            xaxis=dict(title="X", showgrid=True, gridcolor="lightgray"),
            yaxis=dict(title="Y", showgrid=True, gridcolor="lightgray"),
            zaxis=dict(title="Z", showgrid=True, gridcolor="lightgray"),
            bgcolor="white",
            camera=dict(eye=dict(x=2, y=0.5, z=1.8)) 
        ),
        font=dict(family="Arial", size=14, color="black")
    )
    pio.write_image(fig, f"./thesis_imgs/{fig_title}.pdf", format="pdf", width=1200, height=800)
    fig.show()

scatter_torus(points_on_torus, fig_title='Torus_Point_pert0')

scale1 = 0.2
noise1 = np.random.uniform(0, scale1, size=(2000, 3))
pert_torus1 = points_on_torus + noise1
scatter_torus(pert_torus1, fig_title='Torus_Point_pert1')

scale2 = 0.5
noise2 = np.random.uniform(0, scale2, size=(2000, 3))
pert_torus2 = points_on_torus + noise2
scatter_torus(pert_torus2, fig_title='Torus_Point_pert2')

In [ ]:
noise1.max(), noise2.max()

In [ ]:
# Create trace for the point cloud
trace = go.Scatter3d(
    x=points_on_torus[:, 0],
    y=points_on_torus[:, 1],
    z=points_on_torus[:, 2],
    mode='markers',
    marker=dict(
        size=1.5,
        color='red',
        opacity=0.7
    )
)

# Create layout
layout = go.Layout(
    scene=dict(
        xaxis=dict(title='X'),
        yaxis=dict(title='Y'),
        zaxis=dict(title='Z')
    ),
    margin=dict(l=0, r=0, b=0, t=0)
)

# Create figure
fig = go.Figure(data=[trace], layout=layout)

# Show interactive plot
fig.update_layout(
    template="seaborn",  # Alternative to "plotly_white"
    scene=dict(
        xaxis=dict(title="X", showgrid=True, gridcolor="lightgray"),
        yaxis=dict(title="Y", showgrid=True, gridcolor="lightgray"),
        zaxis=dict(title="Z", showgrid=True, gridcolor="lightgray"),
        bgcolor="white",
        camera=dict(eye=dict(x=2, y=0.5, z=1.8)) 
    ),
    font=dict(family="Arial", size=14, color="black")
)
pio.write_image(fig, "./thesis_imgs/TORUS_3d_plot.pdf", format="pdf", width=1200, height=800)
fig.show()

# ALPHA COMPLEXES ON THE POINT CLOUD

In [ ]:
def torus_complexes(torus_points, fig_title): # TORUS Eps xx
    ac = gd.AlphaComplex(points=torus_points) # alpha-complex with alpha=0.005 by default.
    st = ac.create_simplex_tree() # param to modify the default alpha: max_alpha_square = 0.2**2

    print(f'\n>>>>>>>>>>>>>>>>>>>>> { fig_title} <<<<<<<<<<<<<<<<\n')
    print(f"Dimension {st.dimension()}\nNum of simplicies {st.num_simplices()}\nNum of vertices {st.num_vertices()}")

    points = np.array([ac.get_point(i) for i in range(st.num_vertices())])
    print(f"Example of points: \n{points[:3]}")

    # We want to plot triangles of the alpha-complex
    triangles = np.array([s[0] for s in st.get_skeleton(2) if len(s[0]) == 3 and s[1] <= 0.005])
    print(f"Number of triangles {len(triangles)}")

    simplex, filtr_val = next(iter(st.get_simplices()))
    print(f"Example of a simplex: {simplex} with filtration value {filtr_val}")

    BarCodes_AC = st.persistence()
    print(f"Len of persistence points: {len(BarCodes_AC)}")
    print(f"first {BarCodes_AC[0]}\n...\nlast {BarCodes_AC[-1]}\n")

    plt.figure(figsize=(14, 10))
    gd.plot_persistence_diagram(BarCodes_AC)
    plt.title(f"{fig_title} Persistence Diagram", fontsize=20)
    plt.xlabel("Birth", fontsize=16)  # X-axis label
    plt.ylabel("Death", fontsize=16)  # Y-axis label
    plt.savefig(f"./thesis_imgs/{fig_title}_PD.pdf", format="pdf")
    plt.show()

    return st, points, triangles

In [ ]:
torus_st, torus_points, torus_triangles = torus_complexes(points_on_torus, fig_title=f'Torus Eps 0')
torus1_st, torus1_points, torus1_triangles = torus_complexes(pert_torus1, fig_title=f'Torus Eps {scale1}')
torus2_st, torus2_points, torus2_triangles = torus_complexes(pert_torus2, fig_title=f'Torus Eps {scale2}')

## FILTRATIONS

In [ ]:
def filtration(st, points, triangles, fig_title):

    mesh = go.Mesh3d(
        x = points[:, 0], 
        y = points[:, 1], 
        z = points[:, 2], 
        i = triangles[:, 0], 
        j = triangles[:, 1], 
        k = triangles[:, 2]
    )

    fig = go.FigureWidget(
        data = mesh, 
        layout = go.Layout(
            title = dict(
                text = 'Alpha Complex Representation of the 2-Torus'
            ), 
            scene = dict(
                xaxis = dict(nticks = 4, range = [-4., 4.]), 
                yaxis = dict(nticks = 4, range = [-4., 4.]), 
                zaxis = dict(nticks = 4, range = [-2., 2.])
            ),
            margin=dict(l=0, r=0, b=0, t=0)
        )
    )

    for alpha_value in np.array([0.01, 0.03, 0.05, 0.2]): 
        triangles = np.array([s[0] for s in st.get_skeleton(2) if len(s[0]) == 3 and s[1] <= alpha_value])
    
        # Update mesh data
        fig.data[0].i = triangles[:, 0]
        fig.data[0].j = triangles[:, 1]
        fig.data[0].k = triangles[:, 2]
        
        # Save the 2D figure as a PNG (or other formats)
        pio.write_image(fig, f"./thesis_imgs/{fig_title}_AC_{alpha_value:.2f}.pdf", format="pdf", width=1200, height=800)
        
        # Display the updated plot
        iplot(fig)

In [ ]:
filtration(torus_st, torus_points, torus_triangles, fig_title=f'Torus Eps 0')
filtration(torus1_st, torus1_points, torus1_triangles, fig_title=f'Torus Eps {scale1}')
filtration(torus2_st, torus2_points, torus2_triangles, fig_title=f'Torus Eps {scale2}')

## PD DISTANCES

In [ ]:
def st2pers(st, dim):
    pers = st.persistence_intervals_in_dimension(dim)
    if dim ==0:
        pers = pers[:-1] # remove the point [0, infty]
    pers = pers.transpose()
    lengths = pers[1] - pers[0]
    sorted_indices = np.argsort(-lengths)
    sorted_pers = pers[:,sorted_indices].transpose() # now sorted
    return sorted_pers

In [ ]:
# dimension_considered = 2
p = 8
sort = True
limit = 800 # largerst possible value
matrix_pers = np.zeros((3, 3, 3))

for dimension in [0,1,2]:
    print(f'\n>>>>>>>>> {dimension = } ------------')
    ### NO LIMIT AND SORT
    torus_pers = st2pers(torus_st, dimension) # array [limit, 2]
    print(f'{len(torus_pers) = }')
    torus1_pers = st2pers(torus1_st, dimension) # array [limit, 2]
    print(f'{len(torus1_pers) = }')
    torus2_pers = st2pers(torus2_st, dimension) # array [limit, 2]
    print(f'{len(torus2_pers) = }')

    elements = [torus_pers, torus1_pers, torus2_pers]
    for i, pers_i in enumerate(elements):
        for j, pers_j in enumerate(elements):
            dist_ij = wasserstein_distance(pers_i[:limit], pers_j[:limit], order=p, internal_p=1)
            print(f'W_p(comb{i}) = {wasserstein_distance(pers_i, pers_j, order=p, internal_p=1):.4f}, with limt {limit}: {dist_ij:.4f}' )
            matrix_pers[dimension][i][j] = dist_ij


## STABLE RANKS

In [ ]:
preproc = limit # needed for element wise distances
matrix_SR = np.zeros((3, 3, 3))

for dimension in [0,1,2]:
    print(f'\n>>>>>>>>> {dimension = } ------------')
    
    torus_pers = st2pers(torus_st, dimension)[:preproc] # array [limit, 2]
    if len(torus_pers) < preproc:
        torus_pers = np.vstack( (torus_pers, np.zeros((preproc - len(torus_pers), 2))) )

    torus1_pers = st2pers(torus1_st, dimension)[:preproc] # array [limit, 2]
    if len(torus1_pers) < preproc:
        torus1_pers = np.vstack( (torus1_pers, np.zeros((preproc - len(torus1_pers), 2))) )

    torus2_pers = st2pers(torus2_st, dimension)[:preproc] # array [limit, 2]
    if len(torus2_pers) < preproc:
        torus2_pers = np.vstack( (torus2_pers, np.zeros((preproc - len(torus2_pers), 2))) )

    torus_SR = torus_pers[:,1] - torus_pers[:,0]
    torus1_SR = torus1_pers[:,1] - torus1_pers[:,0]
    torus2_SR = torus2_pers[:,1] - torus2_pers[:,0]
    
    plt.figure(figsize=(6, 5))
    sns.lineplot(torus_SR, label = 'SR Torus eps=0.0', marker = 'o')
    plt.title(f"Stablerank of Torus from H{dimension}", fontsize=20, fontweight="bold", pad=12)
    plt.savefig(f"./thesis_imgs/Torus_0_Stableranks_dim{dimension}.pdf", format="pdf")
    plt.show()

    plt.figure(figsize=(6, 5))
    sns.lineplot(torus_SR, label = 'SR Torus eps=0.0', marker = 'o')
    sns.lineplot(torus1_SR, label = 'SR Torus eps=0.2', marker = 's')
    sns.lineplot(torus2_SR, label = 'SR Torus eps=0.5', marker = 'D')
    plt.title(f"Stableranks of Torus from H{dimension}", fontsize=20, fontweight="bold", pad=12)
    plt.savefig(f"./thesis_imgs/Torus_Stableranks_dim{dimension}.pdf", format="pdf")
    plt.show()

    elements = [torus_SR, torus1_SR, torus2_SR]
    
    for i, SR_i in enumerate(elements):
        for j, SR_j in enumerate(elements):
            dist_ij = np.max(np.abs(SR_i - SR_j))
            print(f'IntDist({i}, {j}) = {np.max(np.abs(SR_i - SR_j)):.4f}')
            matrix_SR[dimension][i][j] = dist_ij


In [ ]:
# Element-wise division
torus_Lip_SP = matrix_SR/matrix_pers  # Equivalent to np.multiply(A, B)

for d in [0,1,2]:
    # Plot heatmap
    plt.figure(figsize=(6, 5))
    sns.heatmap(
        torus_Lip_SP[d], 
        annot=True, 
        cmap="coolwarm", 
        fmt=".4f", 
        linewidths=0.5,       # Thicker lines between cells
        linecolor="white",     # White grid lines for clarity
        square=True,  
        cbar=True,        # Square cells for better proportion
        annot_kws={"size": 12}
        )
    plt.title(f"SR/PD from H{d}", fontsize=20, fontweight="bold", pad=12)
    plt.savefig(f"./thesis_imgs/Torus_SR_PD_dim{d}.pdf", format="pdf")
    plt.show()
    print("Result:\n", torus_Lip_SP[d])


## Linf LAYER INIT AND Lip Property

In [ ]:
import torch
# put stable rank inside an l_inf layer to check the lip property
hidden_dim = 10
matrix_NN = np.zeros((3, 3, 3))

params = torch.randn((hidden_dim, preproc)) # randomly generated
p = float('inf')

In [ ]:
for dimension in [0,1,2]:
    print(f'\n>>>>>>>>> {dimension = } ------------')
    
    torus_pers = st2pers(torus_st, dimension)[:preproc] # array [limit, 2]
    if len(torus_pers) < preproc:
        torus_pers = np.vstack( (torus_pers, np.zeros((preproc - len(torus_pers), 2))) )

    torus1_pers = st2pers(torus1_st, dimension)[:preproc] # array [limit, 2]
    if len(torus1_pers) < preproc:
        torus1_pers = np.vstack( (torus1_pers, np.zeros((preproc - len(torus1_pers), 2))) )

    torus2_pers = st2pers(torus2_st, dimension)[:preproc] # array [limit, 2]
    if len(torus2_pers) < preproc:
        torus2_pers = np.vstack( (torus2_pers, np.zeros((preproc - len(torus2_pers), 2))) )

    torus_SR = torus_pers[:,1] - torus_pers[:,0]
    torus1_SR = torus1_pers[:,1] - torus1_pers[:,0]
    torus2_SR = torus2_pers[:,1] - torus2_pers[:,0]

    # NN output
    torus_NN = torch.cdist(torch.tensor(torus_SR, dtype=torch.float).unsqueeze(0), params, p=p)
    torus1_NN = torch.cdist(torch.tensor(torus1_SR, dtype=torch.float).unsqueeze(0), params, p=p)
    torus2_NN = torch.cdist(torch.tensor(torus2_SR, dtype=torch.float).unsqueeze(0), params, p=p)
    # print(torus_NN.shape, torus1_NN.shape, torus2_NN.shape)

    plt.figure(figsize=(6, 5))
    sns.lineplot(torus_NN.squeeze(), label = 'NN Torus eps=0.0', marker = 'o')
    sns.lineplot(torus1_NN.squeeze(), label = 'NN Torus eps=0.2', marker = 's')
    sns.lineplot(torus2_NN.squeeze(), label = 'NN Torus eps=0.5', marker = 'D')
    plt.title(f"NN output of torus from H{dimension}", fontsize=20, fontweight="bold", pad=12)
    plt.savefig(f"./thesis_imgs/Torus_NNoutput_dim{dimension}.pdf", format="pdf")
    plt.show()

    elements = [torus_NN, torus1_NN, torus2_NN]
    
    for i, NN_i in enumerate(elements):
        for j, NN_j in enumerate(elements):
            dist_ij = torch.cdist(NN_i, NN_j, p=p).item()#.detach().numpy()
            print(f'Linf({i}, {j}) = {dist_ij:.4f}')
            matrix_NN[dimension][i][j] = dist_ij


In [ ]:
# Element-wise division
torus_Lip_NS = matrix_NN/matrix_SR  # Equivalent to np.multiply(A, B)

for d in [0,1,2]:
    # Plot heatmap
    plt.figure(figsize=(6, 5))
    sns.heatmap(
        torus_Lip_NS[d], 
        annot=True, 
        cmap="coolwarm", 
        fmt=".4f", 
        linewidths=0.5,       # Thicker lines between cells
        linecolor="white",     # White grid lines for clarity
        square=True,  
        cbar=True,        # Square cells for better proportion
        annot_kws={"size": 12}
        )
    plt.title(f"NN/SR from H{d}", fontsize=20, fontweight="bold", pad=12)
    plt.savefig(f"./thesis_imgs/Torus_NN_SR_dim{d}.pdf", format="pdf")
    plt.show()
    print("Result:\n", torus_Lip_NS[d])


# VIETORIS RIPS ON POINTS CLOUD

In [ ]:

max_len = 0.8 # maximal diameter

skeleton = gd.RipsComplex(points = points_on_torus, max_edge_length = max_len)
# topological graph with:
# as many vertices as there are points;
# as edges only pairs of points whose distance is smaller than or equal to 'max_edge_length.
rips = skeleton.create_simplex_tree(max_dimension = 3)
print(f"{rips.dimension()}, {rips.num_vertices()}, {rips.num_simplices()}") 

In [ ]:
BarCodes_Rips = rips.persistence()
print(f"Len of persistence points: {len(BarCodes_Rips)}")
print(f"first {BarCodes_Rips[0]}\n...\nlast {BarCodes_Rips[-1]}\n")
gd.plot_persistence_diagram(BarCodes_Rips)

In [ ]:
# Example persistence diagram
persistence_diagram = [
    (0, 0.5),  # Birth and death of features
    (1, 2.0),
    (1.5, 2.5)
]

# Convert to GUDHI Persistence Diagram Format
diag = np.array([[b, d] for b, d in persistence_diagram])

# 1. Compute Total Persistence
p = 2  # Use p = 2 for squared total persistence
total_persistence = np.sum([(d - b) ** p for b, d in diag])
print("Total Persistence:", total_persistence)

# 2. Compute Persistence Landscape
landscape = gr.Landscape(resolution=1000)  # Note: "gr.Landscape" is used instead of "PersistenceLandscape"
landscape.fit([diag])
landscape_values = landscape.transform([diag])[0]  # Extract the landscape for the first diagram
print("Persistence Landscape (as array):", landscape_values)

# 3. Compute Persistence Image
persistence_image = gr.PersistenceImage(bandwidth=0.1, weight=lambda x: x[1], resolution=[10, 10])
persistence_image.fit([diag])
image = persistence_image.transform([diag])[0]  # Extract the image for the first diagram
print("Persistence Image (as array):", image)
